In [1]:
SEED=185

In [2]:
import pandas as pd
import numpy as np

from models import modelsRun
from sklearn import preprocessing

In [3]:
DATA='data/mamografias.csv'

# Función para leer los datos
def readData(data_file):
    return pd.read_csv(data_file,sep=',', na_values='?')

data = readData(DATA) # Lectura de los datos

data.rename(columns = {'BI-RADS':'BiRads'}, inplace = True) # Para poder referirnos a esta columna como data.BiRads
data.BiRads.replace(0,pd.NA,inplace=True) # BiRads 0 significa radiografía insuficiente
data.Shape.replace('N',pd.NA,inplace=True) # Lo mismo pasa con Shape N
data.replace(pd.NA, np.nan, inplace=True)

data=data.dropna() # Eliminamos las instancias con valores perdidos
data.shape[0] # Nos quedamos con 825 instancias

# La distribución de la característica densidad es prácticamente degenerada en 3
data.drop('Density', axis='columns',inplace=True)

data.drop('BiRads', axis='columns',inplace=True) # Probamos a eliminar BiRads

In [4]:
# Sklearn necesita datos numéricos (aunque sean nominales)
le = preprocessing.LabelEncoder()
data.Shape = le.fit_transform(data.Shape)
data.Severity = le.fit_transform(data.Severity)
print(le.inverse_transform([0,1])) # Consideraremos maligno como la clase positiva

['benigno' 'maligno']


In [5]:
dataArray=np.array(data)
print(data.columns)
dataArray

Index(['Age', 'Shape', 'Margin', 'Severity'], dtype='object')


array([[67.,  1.,  5.,  1.],
       [58.,  0.,  5.,  1.],
       [28.,  3.,  1.,  0.],
       ...,
       [64.,  0.,  5.,  0.],
       [66.,  0.,  5.,  1.],
       [62.,  1.,  3.,  0.]])

In [6]:
# Separamos en datos y target (label)
x=dataArray[:,:-1]
y=dataArray[:,-1]

In [7]:
# Binarización de los atributos nominales
x_nom=x[:,1:3] # Columnas con variables nominales: Shape y Margin
x=np.delete(x,[1,2],axis=1) # Elimino las columnas
onehot = preprocessing.OneHotEncoder(sparse=False)
x_nom = onehot.fit_transform(x_nom)
x=np.hstack((x,x_nom))

In [8]:
# Estandarizado de las variables
scaler = preprocessing.StandardScaler()
x=scaler.fit_transform(x)
x

array([[ 0.76743452, -0.91287093,  3.03070704, ..., -0.38188131,
        -0.66127319,  2.35533841],
       [ 0.15413118,  1.09544512, -0.32995601, ..., -0.38188131,
        -0.66127319,  2.35533841],
       [-1.89021329, -0.91287093, -0.32995601, ..., -0.38188131,
        -0.66127319, -0.42456744],
       ...,
       [ 0.56300008,  1.09544512, -0.32995601, ..., -0.38188131,
        -0.66127319,  2.35533841],
       [ 0.69928971,  1.09544512, -0.32995601, ..., -0.38188131,
        -0.66127319,  2.35533841],
       [ 0.42671045, -0.91287093,  3.03070704, ...,  2.61861468,
        -0.66127319, -0.42456744]])

In [9]:
modelsRun(x,y,'noBiRads')